In [1]:
import mph as mph
import numpy as np
from scipy.optimize import minimize, differential_evolution
import matplotlib.pyplot as plt
import pandas as pd 
import concurrent.futures
import time

In [3]:
class Optimizer:
    def __init__(self, model_path):
        print("Starting COMSOL client...")
        self.client = mph.start(cores=1)
        self.model = self.client.load(model_path)
        self.optimization_history = []
        self.failed_simulations = 0
        self.max_failures = 3
        self.cache = {}
        
        # Previous optimized values as reference
        self.previous_optimized = {
            'rod_radius': 1.9499, 'rod_length': 41.0417, 'rod_spacing': 6.5633,
            'V_rf': 146.04978, 'V_dc': 25.0164, 'endcap_rad': 7.9028,
            'endcap_thick': 0.9925, 'endcap_offset': 1.9022, 
            'V_endcap': 2.8124, 'f': 8.0736
        }
    
    def smart_parameter_sweep(self):
        """
        Smart sweep that finds best individual parameters and builds narrowed ranges
        """
        print("🔍 Starting Smart Parameter Sweep...")
        
        base_params = list(self.previous_optimized.values())
        param_names = list(self.previous_optimized.keys())
        
        sweep_results = {}
        best_individual_params = base_params.copy()
        best_individual_objective = float('inf')
        
        # Define smart sweep ranges for each parameter
        sweep_config = {
            'rod_radius': {'points': 6, 'range': (0.7, 1.3)},
            'rod_length': {'points': 5, 'range': (0.8, 1.2)},
            'rod_spacing': {'points': 6, 'range': (0.7, 1.3)},
            'V_rf': {'points': 6, 'range': (0.5, 1.5)},
            'V_dc': {'points': 6, 'range': (0.6, 1.4)},
            'endcap_rad': {'points': 5, 'range': (0.8, 1.2)},
            'endcap_thick': {'points': 5, 'range': (0.7, 1.3)},
            'endcap_offset': {'points': 6, 'range': (0.6, 1.4)},
            'V_endcap': {'points': 6, 'range': (0.5, 1.5)},
            'f': {'points': 6, 'range': (0.8, 1.2)}
        }
        
        # Sweep each parameter individually
        for i, param_name in enumerate(param_names):
            print(f"\n Sweeping {param_name}...")
            config = sweep_config[param_name]
            base_value = base_params[i]
            
            low_ratio, high_ratio = config['range']
            values = np.linspace(base_value * low_ratio, base_value * high_ratio, config['points'])
            
            param_results = []
            best_param_value = base_value
            best_param_objective = float('inf')
            
            for value in values:
                test_params = base_params.copy()
                test_params[i] = value
                
                objective = self.fast_objective_function(test_params)
                param_results.append({'value': value, 'objective': objective})
                
                if objective < best_param_objective:
                    best_param_objective = objective
                    best_param_value = value
            
            # Update the best individual parameter
            best_individual_params[i] = best_param_value
            
            sweep_results[param_name] = {
                'values': [r['value'] for r in param_results],
                'objectives': [r['objective'] for r in param_results],
                'best_value': best_param_value,
                'best_objective': best_param_objective
            }
            
            print(f"  Best {param_name}: {best_param_value:.4f} (objective: {best_param_objective:.3f})")
        
        # Build narrowed bounds based on sweep results
        narrowed_bounds = self.build_narrowed_bounds(sweep_results, base_params)
        
        return best_individual_params, narrowed_bounds, sweep_results
    
    def build_narrowed_bounds(self, sweep_results, base_params):
        """
        Build smart narrowed bounds around the best sweep values
        """
        param_names = list(sweep_results.keys())
        narrowed_bounds = []
        
        # Define narrowing factors for each parameter type
        for i, param_name in enumerate(param_names):
            best_value = sweep_results[param_name]['best_value']
            base_value = base_params[i]
            
            # Different narrowing strategies based on parameter type
            if param_name in ['rod_radius', 'rod_spacing', 'endcap_rad', 'endcap_thick', 'endcap_offset']:
                # Geometric parameters - narrow moderately
                narrow_factor = 0.15  # ±15%
            elif param_name in ['V_rf', 'V_dc', 'V_endcap']:
                # Voltage parameters - narrow more aggressively (they have bigger ranges)
                narrow_factor = 0.25  # ±25%
            elif param_name == 'f':
                # Frequency - narrow moderately
                narrow_factor = 0.1   # ±10%
            elif param_name == 'rod_length':
                # Rod length - narrow less (it affects power linearly)
                narrow_factor = 0.1   # ±10%
            else:
                narrow_factor = 0.2   # Default ±20%
            
            lower_bound = max(best_value * (1 - narrow_factor), base_params[i] * 0.5)
            upper_bound = min(best_value * (1 + narrow_factor), base_params[i] * 1.5)
            
            # Ensure bounds are reasonable
            if param_name in ['rod_radius', 'rod_spacing']:
                lower_bound = max(lower_bound, 1.0)
                upper_bound = min(upper_bound, 10.0)
            elif param_name == 'V_rf':
                lower_bound = max(lower_bound, 50)
                upper_bound = min(upper_bound, 500)
            elif param_name == 'f':
                lower_bound = max(lower_bound, 5)
                upper_bound = min(upper_bound, 20)
            
            narrowed_bounds.append((lower_bound, upper_bound))
        
        return narrowed_bounds
    
    def focused_optimization(self, initial_guess, narrowed_bounds):
        """
        Focused optimization within narrowed bounds
        """
        print("\n🎯 Starting Focused Optimization in Narrowed Bounds...")
        
        # Two-step focused optimization
        print("Step 1: Fast local optimization...")
        result_local = minimize(
            self.fast_objective_function,
            initial_guess,
            bounds=narrowed_bounds,
            method='Nelder-Mead',
            options={'maxiter': 8, 'disp': True, 'xatol': 0.05, 'fatol': 0.05}
        )
        
        print("Step 2: Final refinement...")
        result_final = minimize(
            self.fast_objective_function,
            result_local.x,
            bounds=narrowed_bounds,
            method='SLSQP',
            options={'maxiter': 6, 'disp': True, 'ftol': 1e-4}
        )
        
        return result_final
    
    def fast_objective_function(self, params):
        """
        Fast objective function with caching
        """
        params_key = tuple(params)
        if params_key in self.cache:
            return self.cache[params_key]
        
        try:
            self.update_parameters(params)
            self.model.solve()
            metrics = self.extract_trap_metrics()
            
            if metrics is None:
                raise ValueError("Failed to extract metrics")
            
            depth = metrics['depth_eV']
            offset = metrics['offset_mm']
            power = metrics['P_est_mW']
            
            # Balanced objective
            if depth <= 0.01:
                objective = 100.0
            else:
                objective = (
                    0.4 * (depth/ 1.0) +      # Maximize depth
                    0.4 * offset / 0.001 +     # Minimize offset
                    0.2 * power / 100.0        # Minimize power
                )
            
            # Fast penalties
            if offset > 0.1:
                objective += 10.0 * offset
            if power > 500:
                objective += 2.0 * power / 500
            
            self.cache[params_key] = objective
            
            self.optimization_history.append({
                'params': params.copy(),
                'metrics': metrics.copy(),
                'objective': objective
            })
            
            print(f"  Depth: {depth:.3f}eV, Offset: {offset:.3f}mm, Power: {power:.1f}mW")
            
            return objective
            
        except Exception as e:
            print(f"  Simulation failed: {e}")
            self.failed_simulations += 1
            if self.failed_simulations >= self.max_failures:
                raise RuntimeError(f"Too many failures: {self.max_failures}")
            return 100.0
    
    def update_parameters(self, params):
        """Update all parameters in COMSOL"""
        parameter_definitions = [
            ('rod_radius', params[0], 'mm'),
            ('rod_length', params[1], 'mm'),
            ('rod_spacing', params[2], 'mm'),
            ('V_rf', params[3], 'V'),
            ('V_dc', params[4], 'V'),
            ('endcap_rad', params[5], 'mm'),
            ('endcap_thick', params[6], 'mm'),
            ('endcap_offset', params[7], 'mm'),
            ('V_endcap', params[8], 'V'),
            ('f', params[9], 'MHz')
        ]
        
        for param_name, value, unit in parameter_definitions:
            self.model.parameter(param_name, f'{value}[{unit}]')
    
    def extract_trap_metrics(self):
        """Extract key metrics quickly"""
        try:
            return {
                'depth_eV': float(self.model.evaluate('depth_eV')),
                'offset_mm': float(self.model.evaluate('offset_mm')),
                'P_est_mW': float(self.model.evaluate('P_est_mW')),
                'trap_x': float(self.model.evaluate('trap_x')),
                'trap_y': float(self.model.evaluate('trap_y')), 
                'trap_z': float(self.model.evaluate('trap_z'))
            }
        except:
            return None
    
    def get_best_results(self):
        """Get the best optimization results"""
        if not self.optimization_history:
            return None, None
        best_entry = min(self.optimization_history, key=lambda x: x['objective'])
        return best_entry['params'], best_entry['metrics']
    
    def export_optimized_parameters(self, filename='smart_optimized_parameters.txt'):
        """Export final optimized parameters"""
        params, metrics = self.get_best_results()
        if params is None:
            print("No results to export")
            return
            
        param_names = [
            'rod_radius', 'rod_length', 'rod_spacing', 'V_rf', 'V_dc',
            'endcap_rad', 'endcap_thick', 'endcap_offset', 'V_endcap', 'f'
        ]
        units = ['mm', 'mm', 'mm', 'V', 'V', 'mm', 'mm', 'mm', 'V', 'MHz']
        
        with open(filename, 'w') as f:
            for name, value, unit in zip(param_names, params, units):
                f.write(f"{name} {value}[{unit}] \"\"\n")
            
            f.write(f"\n# Performance Metrics:\n")
            f.write(f"# depth_eV: {metrics['depth_eV']:.6f}\n")
            f.write(f"# offset_mm: {metrics['offset_mm']:.6f}\n")
            f.write(f"# P_est_mW: {metrics['P_est_mW']:.2f}\n")
            f.write(f"# trap_x: {metrics['trap_x']:.6f}\n")
            f.write(f"# trap_y: {metrics['trap_y']:.6f}\n")
            f.write(f"# trap_z: {metrics['trap_z']:.6f}\n")
        
        print(f"Optimized parameters exported to {filename}")
        return params, metrics

def main():
    print("OPTIMIZATION")
    print("=" * 60)
    
    start_time = time.time()
    optimizer = Optimizer('3d_pole_trap(1).mph')
    
    try:
        # Step 1: Smart parameter sweep to find promising regions
        print("🔄 STEP 1:Parameter Sweep")
        print("=" * 40)
        best_sweep_params, narrowed_bounds, sweep_results = optimizer.smart_parameter_sweep()
        
        # Step 2: Focused optimization in narrowed ranges
        print("\n🔄 STEP 2: Focused Optimization")
        print("=" * 40)
        result = optimizer.focused_optimization(best_sweep_params, narrowed_bounds)
        
        # Step 3: Export results
        print("\n🔄 STEP 3: Exporting Results")
        print("=" * 40)
        optimized_params, optimized_metrics = optimizer.export_optimized_parameters()
        
        total_time = time.time() - start_time
        total_simulations = len(optimizer.optimization_history)
        
        print("\n" + "="*60)
        print("OPTIMIZATION COMPLETED SUCCESSFULLY!")
        print("="*60)
        print(f"⏱️  Total time: {total_time:.1f} seconds")
        print(f"🔢 Total simulations: {total_simulations}")
        print(f"📈 Final objective: {result.fun:.4f}")
        print(f"🎯 Optimized depth_eV: {optimized_metrics['depth_eV']:.4f}")
        print(f"🎯 Optimized offset_mm: {optimized_metrics['offset_mm']:.4f}")
        print(f"🎯 Optimized power: {optimized_metrics['P_est_mW']:.1f} mW")
        print(f"📍 Trap center: ({optimized_metrics['trap_x']:.3f}, {optimized_metrics['trap_y']:.3f}, {optimized_metrics['trap_z']:.3f}) mm")
        
        # Show parameter improvements
        print(f"\n📊 Parameter improvements from initial:")
        initial_params = list(optimizer.previous_optimized.values())
        param_names = list(optimizer.previous_optimized.keys())
        for i, (name, init, final) in enumerate(zip(param_names, initial_params, optimized_params)):
            change_pct = ((final - init) / init) * 100
            print(f"  {name:15}: {init:8.3f} → {final:8.3f} ({change_pct:+.1f}%)")
        
    except Exception as e:
        print(f"\n Optimization interrupted: {e}")
        try:
            optimizer.export_optimized_parameters('smart_optimization_partial.txt')
        except:
            pass
    
    finally:
        print("\n Cleaning up...")
        optimizer.client.disconnect()

if __name__ == "__main__":
    main()

🧠 SMART SWEEP + FOCUSED OPTIMIZATION
Starting COMSOL client...
🔄 STEP 1: Smart Parameter Sweep
🔍 Starting Smart Parameter Sweep...

📊 Sweeping rod_radius...
  Depth: 1.050eV, Offset: 10.000mm, Power: 830.4mW
  Depth: 0.738eV, Offset: 10.000mm, Power: 830.4mW
  Depth: 0.560eV, Offset: 10.000mm, Power: 830.4mW
  Depth: 0.415eV, Offset: 10.000mm, Power: 830.4mW
  Depth: 0.296eV, Offset: 10.000mm, Power: 830.4mW
  Depth: 0.205eV, Offset: 10.000mm, Power: 830.4mW
  Best rod_radius: 2.5349 (objective: 4105.065)

📊 Sweeping rod_length...
  Depth: 5.112eV, Offset: 30.000mm, Power: 531.4mW
  Depth: 1.093eV, Offset: 30.000mm, Power: 672.6mW
  Depth: 0.487eV, Offset: 10.000mm, Power: 830.4mW
  Depth: 0.506eV, Offset: 10.000mm, Power: 1004.7mW
  Depth: 0.513eV, Offset: 10.000mm, Power: 1195.7mW
  Best rod_length: 41.0417 (objective: 4105.177)

📊 Sweeping rod_spacing...
  Depth: 0.024eV, Offset: 10.000mm, Power: 1694.6mW
  Depth: 0.124eV, Offset: 10.000mm, Power: 1234.9mW
  Depth: 0.333eV, Offset: 

/tmp/ipykernel_617687/990249813.py:141: RuntimeWarning: Maximum number of iterations has been exceeded.
  result_local = minimize(


  Depth: 0.344eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.344eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.347eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.343eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.343eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.345eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.343eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.344eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.343eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.343eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.604eV, Offset: 30.000mm, Power: 72.0mW
  Depth: 0.305eV, Offset: 30.000mm, Power: 70.6mW
  Depth: 0.319eV, Offset: 10.000mm, Power: 70.0mW
  Depth: 0.331eV, Offset: 10.000mm, Power: 69.8mW
  Depth: 0.336eV, Offset: 10.000mm, Power: 69.7mW
  Depth: 0.342eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.340eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.341eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.348eV, Offset: 10.000mm, Power: 69.6mW
  Depth: 0.344eV, Offset: 10.000mm, Power: 69.6mW
